In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import os
import pickle

from sklearn.linear_model import Lasso, LassoCV
from sklearn.model_selection import cross_val_score

In [11]:
# --- Step 1: Load the datasets ---
print("="*60)
print("MACHINE LEARNING ASSIGNMENT - LASSO REGRESSION")
print("="*60)

try:
    # Reading from the Hotel-Property-Value-Dataset folder
    train_df = pd.read_csv('../Hotel-Property-Value-Dataset/train.csv')
    test_df = pd.read_csv('../Hotel-Property-Value-Dataset/test.csv')
    sample_submission_df = pd.read_csv('../Hotel-Property-Value-Dataset/sample_submission.csv')
    print("✅ Files loaded successfully!")
    print(f"Training data shape: {train_df.shape}")
    print(f"Test data shape: {test_df.shape}")
except FileNotFoundError as e:
    print(f"❌ Error: {e}")
    print("Ensure the Hotel-Property-Value-Dataset folder contains train.csv, test.csv, and sample_submission.csv")
    exit()

# --- Target Variable ---
TARGET_VARIABLE = "HotelValue"

if TARGET_VARIABLE not in train_df.columns:
    print(f"❌ Error: The target column '{TARGET_VARIABLE}' was not found in train.csv.")
    print(f"Available columns are: {list(train_df.columns)}")
    exit()

print(f"\n📊 Target variable: {TARGET_VARIABLE}")
print(f"Target statistics:\n{train_df[TARGET_VARIABLE].describe()}")

MACHINE LEARNING ASSIGNMENT - LASSO REGRESSION
✅ Files loaded successfully!
Training data shape: (1200, 81)
Test data shape: (260, 80)

📊 Target variable: HotelValue
Target statistics:
count      1200.000000
mean     181709.895833
std       77638.660223
min       34900.000000
25%      130000.000000
50%      165000.000000
75%      215000.000000
max      745000.000000
Name: HotelValue, dtype: float64


In [12]:
# --- Step 2: Data Preprocessing (Course Concepts) ---
print(f"\n" + "="*60)
print("STEP 2: DATA PREPROCESSING")
print("="*60)

# Separate features (X) from the target (y)
X_train_full = train_df.drop([TARGET_VARIABLE], axis=1)
y_train = train_df[TARGET_VARIABLE]
X_test_full = test_df.copy()

print(f"Features in training data: {X_train_full.shape[1]}")
print(f"Features in test data: {X_test_full.shape[1]}")

# Identify numeric and categorical columns
numeric_features = X_train_full.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_train_full.select_dtypes(include=['object']).columns.tolist()

print(f"\n📈 Numeric features ({len(numeric_features)}): {numeric_features[:5]}..." if len(numeric_features) > 5 else f"\n📈 Numeric features ({len(numeric_features)}): {numeric_features}")
print(f"📝 Categorical features ({len(categorical_features)}): {categorical_features[:5]}..." if len(categorical_features) > 5 else f"📝 Categorical features ({len(categorical_features)}): {categorical_features}")


STEP 2: DATA PREPROCESSING
Features in training data: 80
Features in test data: 80

📈 Numeric features (37): ['Id', 'PropertyClass', 'RoadAccessLength', 'LandArea', 'OverallQuality']...
📝 Categorical features (43): ['ZoningCategory', 'RoadType', 'ServiceLaneType', 'PlotShape', 'LandElevation']...


In [13]:
# --- Step 3: Handle Missing Values and Encode Categorical Variables ---

# Configuration: Set your preferred cache directory here
CACHE_DIR = '../Hotel-Property-Value-Dataset/'  # Change this path as needed

def save_cleaned_data(X_train_processed, y_train, base_path=CACHE_DIR):
    """Save cleaned data as CSV file"""
    os.makedirs(base_path, exist_ok=True)
    
    # Combine features and target into single dataframe
    cleaned_df = X_train_processed.copy()
    cleaned_df['HotelValue'] = y_train
    
    # Save as CSV
    csv_path = os.path.join(base_path, 'train_cleaned_mada.csv')
    cleaned_df.to_csv(csv_path, index=False)
    
    print(f"✅ Cleaned data saved to {csv_path}")
    print(f"   Shape: {cleaned_df.shape}")
    return csv_path

def load_cleaned_data(base_path=CACHE_DIR):
    """Load cleaned data if it exists"""
    csv_path = os.path.join(base_path, 'train_cleaned_mada.csv')
    
    if os.path.exists(csv_path):
        cleaned_df = pd.read_csv(csv_path)
        
        # Separate features and target
        X_train_processed = cleaned_df.drop('HotelValue', axis=1)
        y_train = cleaned_df['HotelValue']
        
        print(f"✅ Cleaned data loaded from {csv_path}")
        print(f"   Shape: {cleaned_df.shape}")
        return X_train_processed, y_train, True
    else:
        print("❌ Cleaned data not found. Will perform preprocessing...")
        return None, None, False

def preprocess_data(X_train, X_test, numeric_features, categorical_features):
    """
    Preprocess the data by handling missing values and encoding categorical variables
    Following course preprocessing concepts
    """
    X_train_processed = X_train.copy()
    X_test_processed = X_test.copy()
    
    # Handle numeric features - Fill with median (robust to outliers)
    for col in numeric_features:
        if col in X_train_processed.columns:
            median_val = X_train_processed[col].median()
            X_train_processed[col] = X_train_processed[col].fillna(median_val)
            X_test_processed[col] = X_test_processed[col].fillna(median_val)
    
    # Handle categorical features - Label encoding
    label_encoders = {}
    for col in categorical_features:
        if col in X_train_processed.columns:
            # Fill missing values with mode (most frequent value)
            mode_val = X_train_processed[col].mode()[0] if not X_train_processed[col].mode().empty else 'Unknown'
            X_train_processed[col] = X_train_processed[col].fillna(mode_val)
            X_test_processed[col] = X_test_processed[col].fillna(mode_val)
            
            # Label encode categorical variables
            le = LabelEncoder()
            # Fit on combined data to handle unseen categories in test set
            combined_data = pd.concat([X_train_processed[col], X_test_processed[col]], axis=0)
            le.fit(combined_data)
            
            X_train_processed[col] = le.transform(X_train_processed[col])
            X_test_processed[col] = le.transform(X_test_processed[col])
            label_encoders[col] = le
    
    return X_train_processed, X_test_processed, label_encoders

# Try to load cleaned data first
X_train_processed, y_train_series, data_loaded = load_cleaned_data()

if not data_loaded:
    print("🔄 Starting data preprocessing...")
    
    # Preprocess the data
    X_train_processed, X_test_processed, label_encoders = preprocess_data(
        X_train_full, X_test_full, numeric_features, categorical_features
    )

    print(f"\nAfter preprocessing:")
    print(f"✅ Training data shape: {X_train_processed.shape}")
    print(f"✅ Test data shape: {X_test_processed.shape}")
    print(f"✅ Missing values in training data: {X_train_processed.isnull().sum().sum()}")
    print(f"✅ Missing values in test data: {X_test_processed.isnull().sum().sum()}")

    # Save cleaned training data as CSV
    y_train_for_saving = y_train.values if hasattr(y_train, 'values') else y_train
    save_cleaned_data(X_train_processed, y_train_for_saving)
    
    # Convert y_train to pandas Series for consistency
    y_train_series = pd.Series(y_train_for_saving, name='HotelValue')
    
else:
    print(f"✅ Using cleaned data from CSV!")
    print(f"Training features shape: {X_train_processed.shape}")
    print(f"Target shape: {y_train_series.shape}")
    
    # Still need to preprocess test data when loading from CSV
    print("🔄 Processing test data...")
    _, X_test_processed, label_encoders = preprocess_data(
        X_train_full, X_test_full, numeric_features, categorical_features
    )

# Feature Scaling (Standardization)
print(f"\n" + "="*60)
print("STEP 3: FEATURE SCALING")
print("="*60)

# Scale all features using standardization (mean=0, std=1)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_processed)
X_test_scaled = scaler.transform(X_test_processed)

print(f"✅ Features scaled successfully!")
print(f"Training features shape: {X_train_scaled.shape}")
print(f"Test features shape: {X_test_scaled.shape}")
print(f"Feature means after scaling: {np.mean(X_train_scaled, axis=0)[:5]} (should be ~0)")
print(f"Feature stds after scaling: {np.std(X_train_scaled, axis=0)[:5]} (should be ~1)")

# Convert to numpy array for mathematical operations
y_train = y_train_series.values


✅ Cleaned data loaded from ../Hotel-Property-Value-Dataset/train_cleaned_mada.csv
   Shape: (1200, 81)
✅ Using cleaned data from CSV!
Training features shape: (1200, 80)
Target shape: (1200,)
🔄 Processing test data...

STEP 3: FEATURE SCALING
✅ Features scaled successfully!
Training features shape: (1200, 80)
Test features shape: (260, 80)
Feature means after scaling: [-9.47390314e-17  1.77635684e-17  2.54611147e-16 -7.69754630e-17
 -6.21724894e-17] (should be ~0)
Feature stds after scaling: [1. 1. 1. 1. 1.] (should be ~1)


In [24]:
# === STEP 4: LASSO REGRESSION USING SCIKIT-LEARN ===
# Using sklearn's Lasso for efficiency and built-in cross-validation

print(f"\n" + "="*60)
print("STEP 4: LASSO REGRESSION - SCIKIT-LEARN IMPLEMENTATION")
print("="*60)


print("📐 Using sklearn LASSO Regression:")
print("Objective: Minimize ||Xθ - y||² + λ||θ||₁")
print("Key Feature: Automatic feature selection via L1 regularization")
print(f"Training data shape: {X_train_scaled.shape}")

# LASSO regression hyperparameter tuning
alpha_values = [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 5000.0]

print(f"\n🔍 HYPERPARAMETER TUNING - TESTING α VALUES:")
print("-" * 70)

# Use LassoCV for automatic cross-validation
lasso_cv = LassoCV(alphas=alpha_values, cv=5, random_state=42, max_iter=10000)
lasso_cv.fit(X_train_scaled, y_train)

best_alpha = lasso_cv.alpha_
print(f"✅ Best α (regularization) found via CV: {best_alpha}")

# Train final model with best alpha
model = Lasso(alpha=best_alpha, max_iter=2000, random_state=42)
model.fit(X_train_scaled, y_train)

# Test different alpha values for comparison
all_results = {}
for alpha in alpha_values:
    lasso_temp = Lasso(alpha=alpha, max_iter=2000, random_state=42)
    lasso_temp.fit(X_train_scaled, y_train)
    r2_score = lasso_temp.score(X_train_scaled, y_train)
    
    # Calculate sparsity metrics
    zero_weights = np.sum(np.abs(lasso_temp.coef_) < 1e-6)
    active_features = len(lasso_temp.coef_) - zero_weights
    sparsity_percent = (zero_weights / len(lasso_temp.coef_)) * 100
    
    # L1 regularization penalty
    reg_penalty = alpha * np.sum(np.abs(lasso_temp.coef_))
    
    all_results[alpha] = {
        'model': lasso_temp,
        'r2': r2_score,
        'reg_penalty': reg_penalty,
        'zero_weights': zero_weights,
        'active_features': active_features,
        'sparsity': sparsity_percent,
        'coef_norm': np.sum(np.abs(lasso_temp.coef_))
    }
    
    print(f"α = {alpha:8.1f} | R² = {r2_score:.4f} | Active = {active_features:2d}/{len(lasso_temp.coef_)} | Sparsity = {sparsity_percent:5.1f}% | L1 = {np.sum(np.abs(lasso_temp.coef_)):6.2f}")

# Get results for best model
best_results = all_results[best_alpha]

print(f"\n✅ OPTIMAL LASSO REGRESSION MODEL:")
print("-" * 40)
print(f"Best α (regularization): {best_alpha}")
print(f"Training R² score: {model.score(X_train_scaled, y_train):.4f}")
print(f"Number of features: {X_train_scaled.shape[1]}")
print(f"Active features: {best_results['active_features']}/{len(model.coef_)}")
print(f"Sparsity: {best_results['sparsity']:.1f}% of features set to zero")
print(f"Bias term (intercept): {model.intercept_:.2f}")
print(f"Non-zero coefficients (first 5): {model.coef_[np.abs(model.coef_) > 1e-6][:5]}")
print(f"Coefficient L1 norm: {np.sum(np.abs(model.coef_)):.2f}")

# Make predictions
train_predictions = model.predict(X_train_scaled)
test_predictions = model.predict(X_test_scaled)

# Feature selection analysis
print(f"\n📊 FEATURE SELECTION ANALYSIS:")
print("-" * 40)
nonzero_indices = np.where(np.abs(model.coef_) > 1e-6)[0]
zero_indices = np.where(np.abs(model.coef_) <= 1e-6)[0]

print(f"Features selected by LASSO: {len(nonzero_indices)}")
print(f"Features eliminated by LASSO: {len(zero_indices)}")
print(f"Feature selection ratio: {len(nonzero_indices)/len(model.coef_)*100:.1f}%")

# Compare regularization strengths
print(f"\n📈 REGULARIZATION PATH ANALYSIS:")
print("-" * 50)
print("α        | R²      | Active | Sparsity | L1 Norm")
print("-" * 50)
for alpha in sorted(all_results.keys()):
    res = all_results[alpha]
    print(f"{alpha:8.1f} | {res['r2']:.4f} |   {res['active_features']:2d}   |  {res['sparsity']:5.1f}% | {res['coef_norm']:7.2f}")

# Store coefficients for analysis (including intercept)
weights = np.concatenate([[model.intercept_], model.coef_])


STEP 4: LASSO REGRESSION - SCIKIT-LEARN IMPLEMENTATION
📐 Using sklearn LASSO Regression:
Objective: Minimize ||Xθ - y||² + λ||θ||₁
Key Feature: Automatic feature selection via L1 regularization
Training data shape: (1200, 80)

🔍 HYPERPARAMETER TUNING - TESTING α VALUES:
----------------------------------------------------------------------
✅ Best α (regularization) found via CV: 5000.0


/Users/hemanthmada/vscodeProjects/ml_assignment_1/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.585e+09, tolerance: 7.227e+08
  model = cd_fast.enet_coordinate_descent(


α =      0.0 | R² = 0.8675 | Active = 80/80 | Sparsity =   0.0% | L1 = 242187.98


/Users/hemanthmada/vscodeProjects/ml_assignment_1/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.588e+09, tolerance: 7.227e+08
  model = cd_fast.enet_coordinate_descent(


α =      0.1 | R² = 0.8675 | Active = 80/80 | Sparsity =   0.0% | L1 = 241915.78


/Users/hemanthmada/vscodeProjects/ml_assignment_1/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.612e+09, tolerance: 7.227e+08
  model = cd_fast.enet_coordinate_descent(


α =      1.0 | R² = 0.8675 | Active = 79/80 | Sparsity =   1.2% | L1 = 240441.15
α =     10.0 | R² = 0.8675 | Active = 78/80 | Sparsity =   2.5% | L1 = 235433.79
α =    100.0 | R² = 0.8673 | Active = 74/80 | Sparsity =   7.5% | L1 = 219356.68
α =   1000.0 | R² = 0.8578 | Active = 48/80 | Sparsity =  40.0% | L1 = 158683.84
α =   5000.0 | R² = 0.8072 | Active = 21/80 | Sparsity =  73.8% | L1 = 93342.31

✅ OPTIMAL LASSO REGRESSION MODEL:
----------------------------------------
Best α (regularization): 5000.0
Training R² score: 0.8072
Number of features: 80
Active features: 21/80
Sparsity: 73.8% of features set to zero
Bias term (intercept): 181709.90
Non-zero coefficients (first 5): [-2784.31349629  2085.63731527 22065.37810362  2353.15126429
   911.78349076]
Coefficient L1 norm: 93342.31

📊 FEATURE SELECTION ANALYSIS:
----------------------------------------
Features selected by LASSO: 21
Features eliminated by LASSO: 59
Feature selection ratio: 26.2%

📈 REGULARIZATION PATH ANALYSIS:
--

/Users/hemanthmada/vscodeProjects/ml_assignment_1/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.811e+09, tolerance: 7.227e+08
  model = cd_fast.enet_coordinate_descent(


In [25]:
# === STEP 5: ERROR FUNCTION ANALYSIS ===
# Following course concepts: Multiple Error Functions for Model Evaluation

print(f"\n" + "="*60)
print("STEP 5: ERROR FUNCTION ANALYSIS")
print("="*60)

def calculate_error_functions(y_true, y_pred):
    """
    Calculate multiple error functions as taught in course
    """
    n = len(y_true)
    
    # 1. Mean Squared Error (MSE) - L2 Loss
    mse = np.mean((y_true - y_pred) ** 2)
    
    # 2. Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    
    # 3. Mean Absolute Error (MAE) - L1 Loss
    mae = np.mean(np.abs(y_true - y_pred))
    
    # 4. R-squared (Coefficient of Determination)
    ss_res = np.sum((y_true - y_pred) ** 2)  # Residual sum of squares
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)  # Total sum of squares
    r2 = 1 - (ss_res / ss_tot)
    
    # 5. Adjusted R-squared
    n_features = X_train_scaled.shape[1]
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - n_features - 1)
    
    # 6. Mean Absolute Percentage Error (MAPE)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'Adjusted R²': adj_r2,
        'MAPE': mape
    }

# Calculate error functions for training data
train_errors = calculate_error_functions(y_train, train_predictions)

print("📊 TRAINING SET ERROR ANALYSIS:")
print("-" * 40)
for metric, value in train_errors.items():
    if metric in ['MAPE']:
        print(f"{metric:12}: {value:.2f}%")
    elif metric in ['MSE']:
        print(f"{metric:12}: {value:,.0f}")
    elif metric in ['RMSE', 'MAE']:
        print(f"{metric:12}: ${value:,.2f}")
    else:
        print(f"{metric:12}: {value:.4f}")

# Analysis of prediction quality
print(f"\n📈 PREDICTION QUALITY ANALYSIS:")
print("-" * 40)
residuals = y_train - train_predictions
print(f"Mean residual: ${np.mean(residuals):,.2f}")
print(f"Std of residuals: ${np.std(residuals):,.2f}")
print(f"Min prediction: ${np.min(train_predictions):,.2f}")
print(f"Max prediction: ${np.max(train_predictions):,.2f}")
print(f"Predictions in range [0, max_actual]: {np.sum((train_predictions >= 0) & (train_predictions <= np.max(y_train))) / len(train_predictions) * 100:.1f}%")


STEP 5: ERROR FUNCTION ANALYSIS
📊 TRAINING SET ERROR ANALYSIS:
----------------------------------------
MSE         : 1,161,282,608
RMSE        : $34,077.60
MAE         : $20,767.77
R²          : 0.8072
Adjusted R² : 0.7934
MAPE        : 12.12%

📈 PREDICTION QUALITY ANALYSIS:
----------------------------------------
Mean residual: $-0.00
Std of residuals: $34,077.60
Min prediction: $27,439.93
Max prediction: $542,309.12
Predictions in range [0, max_actual]: 100.0%


In [26]:
# === STEP 6: MODEL DIAGNOSTICS AND VALIDATION ===
# Following course concepts: Model Validation and Diagnostics

print(f"\n" + "="*60)
print("STEP 6: MODEL DIAGNOSTICS AND VALIDATION")
print("="*60)

# 1. Prediction Examples (Sample Analysis)
print("🔍 PREDICTION EXAMPLES (First 10 samples):")
print("-" * 50)
comparison_df = pd.DataFrame({
    'Actual': y_train[:10],
    'Predicted': train_predictions[:10],
    'Residual': y_train[:10] - train_predictions[:10],
    'Abs_Error': np.abs(y_train[:10] - train_predictions[:10])
})
print(comparison_df.round(2))

# 2. Model Complexity Analysis
print(f"\n📊 MODEL COMPLEXITY ANALYSIS:")
print("-" * 40)
print(f"Number of training samples: {len(y_train)}")
print(f"Number of features: {X_train_scaled.shape[1]}")
print(f"Parameters to data ratio: {len(weights)}/{len(y_train)} = {len(weights)/len(y_train):.4f}")

# 3. Weight Analysis
print(f"\n⚖️ LEARNED PARAMETERS ANALYSIS:")
print("-" * 40)
print(f"Intercept (bias): {weights[0]:.4f}")
print(f"Largest positive weight: {np.max(weights[1:]):.4f}")
print(f"Largest negative weight: {np.min(weights[1:]):.4f}")
print(f"Weight standard deviation: {np.std(weights[1:]):.4f}")

# 4. Residual Analysis
print(f"\n📉 RESIDUAL ANALYSIS:")
print("-" * 40)
residuals = y_train - train_predictions
print(f"Residual mean (should be ~0): {np.mean(residuals):.4f}")
print(f"Residual std: {np.std(residuals):.2f}")
print(f"Residual skewness: {np.mean(((residuals - np.mean(residuals)) / np.std(residuals)) ** 3):.4f}")

# 5. Prediction Bounds Analysis
negative_predictions = np.sum(train_predictions < 0)
if negative_predictions > 0:
    print(f"\n⚠️ WARNING: {negative_predictions} negative predictions detected!")
    print("This suggests the model may need constraints or regularization.")

print(f"\n✅ MODEL VALIDATION COMPLETE!")


STEP 6: MODEL DIAGNOSTICS AND VALIDATION
🔍 PREDICTION EXAMPLES (First 10 samples):
--------------------------------------------------
     Actual  Predicted   Residual  Abs_Error
0  395000.0  269935.31  125064.69  125064.69
1  165000.0  179562.73  -14562.73   14562.73
2  128200.0  116618.86   11581.14   11581.14
3  275000.0  231008.74   43991.26   43991.26
4  311872.0  291838.00   20034.00   20034.00
5  214000.0  217473.11   -3473.11    3473.11
6  153500.0  187667.42  -34167.42   34167.42
7  144000.0  163986.41  -19986.41   19986.41
8  115000.0  125787.62  -10787.62   10787.62
9  180000.0  183519.48   -3519.48    3519.48

📊 MODEL COMPLEXITY ANALYSIS:
----------------------------------------
Number of training samples: 1200
Number of features: 80
Parameters to data ratio: 81/1200 = 0.0675

⚖️ LEARNED PARAMETERS ANALYSIS:
----------------------------------------
Intercept (bias): 181709.8958
Largest positive weight: 22065.3781
Largest negative weight: -6759.5807
Weight standard deviatio

In [27]:
# === STEP 7: MODEL ANALYSIS ===
# Analysis of the sklearn LASSO Regression model

print(f"\n" + "="*60)
print("STEP 7: LASSO MODEL ANALYSIS")
print("="*60)

# 1. LASSO Regression Model Properties
print("📐 LASSO REGRESSION MODEL PROPERTIES:")
print("-" * 50)
print(f"Optimal α (regularization): {model.alpha}")
print(f"Intercept: {model.intercept_:.4f}")
print(f"Number of coefficients: {len(model.coef_)}")
print(f"Coefficient L1 norm: {np.sum(np.abs(model.coef_)):.4f}")
print(f"Number of zero coefficients: {np.sum(np.abs(model.coef_) < 1e-6)}")
print(f"Number of active features: {np.sum(np.abs(model.coef_) >= 1e-6)}")
print(f"Sparsity percentage: {np.sum(np.abs(model.coef_) < 1e-6)/len(model.coef_)*100:.1f}%")

# 2. Model Performance Metrics
print(f"\n📊 MODEL PERFORMANCE:")
print("-" * 50)
train_r2 = model.score(X_train_scaled, y_train)
print(f"Training R² score: {train_r2:.4f}")
print(f"Training R² percentage: {train_r2*100:.2f}%")

# Cross-validation score
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
print(f"Cross-validation R² mean: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

# 3. Feature Selection Analysis
print(f"\n🎯 FEATURE SELECTION ANALYSIS:")
print("-" * 50)
active_mask = np.abs(model.coef_) >= 1e-6
zero_mask = np.abs(model.coef_) < 1e-6

if hasattr(X_train_processed, 'columns'):
    feature_names = X_train_processed.columns
    
    # Active features
    active_features_df = pd.DataFrame({
        'Feature': feature_names[active_mask],
        'Coefficient': model.coef_[active_mask],
        'Abs_Coefficient': np.abs(model.coef_[active_mask])
    }).sort_values('Abs_Coefficient', ascending=False)
    
    print("TOP 10 SELECTED FEATURES:")
    print(active_features_df.head(10).to_string(index=False))
    
    # Zero features (eliminated)
    zero_features = feature_names[zero_mask]
    print(f"\nFEATURES ELIMINATED BY LASSO: {len(zero_features)}")
    if len(zero_features) <= 10:
        print(f"Eliminated features: {list(zero_features)}")
    else:
        print(f"First 10 eliminated: {list(zero_features[:10])}")
else:
    # If we don't have feature names, show indices
    active_indices = np.where(active_mask)[0]
    zero_indices = np.where(zero_mask)[0]
    
    print("TOP 10 SELECTED FEATURE INDICES:")
    coef_indices = np.argsort(np.abs(model.coef_[active_mask]))[::-1][:10]
    for i, idx in enumerate(coef_indices):
        feature_idx = active_indices[idx]
        print(f"Feature {feature_idx:2d}: {model.coef_[feature_idx]:8.4f}")
    
    print(f"\nFEATURES ELIMINATED: {len(zero_indices)} features")

# 4. Regularization Path Analysis
print(f"\n📈 REGULARIZATION PATH ANALYSIS:")
print("-" * 55)
print("α        | R²      | Active | Sparsity | L1 Norm")
print("-" * 55)
for alpha in sorted(all_results.keys()):
    result = all_results[alpha]
    print(f"{alpha:8.1f} | {result['r2']:7.4f} | {result['active_features']:6d} | {result['sparsity']:7.1f}% | {result['coef_norm']:7.2f}")

# 5. Model Complexity Analysis
print(f"\n🔢 MODEL COMPLEXITY:")
print("-" * 50)
n_samples = len(y_train)
n_params = len(model.coef_) + 1  # +1 for intercept
active_params = np.sum(np.abs(model.coef_) >= 1e-6) + 1  # +1 for intercept
df_residual = n_samples - active_params

print(f"Number of samples: {n_samples}")
print(f"Total parameters: {n_params}")
print(f"Active parameters: {active_params}")
print(f"Effective degrees of freedom: {df_residual}")
print(f"Parameter/Sample ratio: {n_params/n_samples:.3f}")
print(f"Active parameter/Sample ratio: {active_params/n_samples:.3f}")

# 6. Sparsity Benefits Assessment
print(f"\n⚖️ SPARSITY BENEFITS:")
print("-" * 50)
sparsity_ratio = np.sum(np.abs(model.coef_) < 1e-6) / len(model.coef_)
reg_penalty = model.alpha * np.sum(np.abs(model.coef_))
data_fit = np.mean((y_train - train_predictions) ** 2)

print(f"Sparsity ratio: {sparsity_ratio*100:.1f}%")
print(f"Model interpretability: {'High' if sparsity_ratio > 0.5 else 'Medium' if sparsity_ratio > 0.2 else 'Low'}")
print(f"Regularization penalty: {reg_penalty:,.2f}")
print(f"Data fit term (MSE): {data_fit:,.0f}")
print(f"Total objective value: {data_fit + reg_penalty:,.0f}")

# Assessment of regularization strength
if model.alpha < 1.0:
    print("✅ Light regularization - minimal feature selection")
elif model.alpha < 100.0:
    print("⚖️ Moderate regularization - balanced feature selection")
else:
    print("🔍 Strong regularization - aggressive feature selection")

# 7. Feature Selection Quality
print(f"\n🏆 FEATURE SELECTION QUALITY:")
print("-" * 50)
if np.sum(active_mask) > 0:
    avg_coef_magnitude = np.mean(np.abs(model.coef_[active_mask]))
    max_coef_magnitude = np.max(np.abs(model.coef_))
    min_active_coef = np.min(np.abs(model.coef_[active_mask]))
    
    print(f"Average active coefficient magnitude: {avg_coef_magnitude:.4f}")
    print(f"Maximum coefficient magnitude: {max_coef_magnitude:.4f}")
    print(f"Minimum active coefficient magnitude: {min_active_coef:.6f}")
    print(f"Coefficient range (active): {max_coef_magnitude/min_active_coef:.1f}x")
    
    print("✅ LASSO successfully performed automatic feature selection")
else:
    print("⚠️ All features eliminated - consider reducing regularization")


STEP 7: LASSO MODEL ANALYSIS
📐 LASSO REGRESSION MODEL PROPERTIES:
--------------------------------------------------
Optimal α (regularization): 5000.0
Intercept: 181709.8958
Number of coefficients: 80
Coefficient L1 norm: 93342.3083
Number of zero coefficients: 59
Number of active features: 21
Sparsity percentage: 73.8%

📊 MODEL PERFORMANCE:
--------------------------------------------------
Training R² score: 0.8072
Training R² percentage: 80.72%
Cross-validation R² mean: 0.7754 ± 0.0522

🎯 FEATURE SELECTION ANALYSIS:
--------------------------------------------------
TOP 10 SELECTED FEATURES:
            Feature  Coefficient  Abs_Coefficient
     OverallQuality 22065.378104     22065.378104
         UsableArea 19904.963637     19904.963637
    ParkingCapacity  7273.306611      7273.306611
     BasementHeight -6759.580707      6759.580707
     KitchenQuality -6355.054174      6355.054174
    ExteriorQuality -4848.105726      4848.105726
BasementFacilitySF1  4119.289042      4119.289

In [28]:
# === STEP 8: KAGGLE SUBMISSION PREPARATION ===
# Final step: Prepare submission file for Kaggle competition

print(f"\n" + "="*60)
print("STEP 8: KAGGLE SUBMISSION PREPARATION")
print("="*60)

# Load test data and make predictions
print("📁 LOADING TEST DATA AND MAKING PREDICTIONS:")
print("-" * 50)

# Test data was already preprocessed and scaled in Step 3
print(f"Test data shape: {X_test_scaled.shape}")

# Make predictions using our trained sklearn LASSO model
test_predictions = model.predict(X_test_scaled)

print(f"✅ Test predictions generated for {len(test_predictions)} samples")
print(f"Prediction range: [{test_predictions.min():.2f}, {test_predictions.max():.2f}]")

# Create submission file using actual test IDs
submission_df = pd.DataFrame({
    'Id': test_df['Id'].values,  # Use actual IDs from test dataset
    'HotelValue': test_predictions  # Use HotelValue as per sample submission format
})

# Save submission file
submission_path = '/Users/hemanthmada/vscodeProjects/ml_assignment_1/submissions/3_LASSO_regression_sklearn.csv'
submission_df.to_csv(submission_path, index=False)

print(f"📤 SUBMISSION FILE CREATED:")
print("-" * 50)
print(f"File saved at: {submission_path}")
print(f"Submission shape: {submission_df.shape}")
print("\nFirst 5 predictions:")
print(submission_df.head())

# Final summary of the entire LASSO regression implementation
print(f"\n" + "="*60)
print("🎓 SKLEARN LASSO REGRESSION SUMMARY")
print("="*60)
print("✅ Step 1: Data Loading and Exploration - COMPLETED")
print("✅ Step 2: Data Preprocessing and Feature Engineering - COMPLETED")
print("✅ Step 3: Feature Scaling - COMPLETED")
print("✅ Step 4: Sklearn LASSO Regression with CV - COMPLETED")
print("✅ Step 5: Comprehensive Error Function Analysis - COMPLETED")
print("✅ Step 6: Model Diagnostics and Validation - COMPLETED")
print("✅ Step 7: LASSO Model Analysis - COMPLETED")
print("✅ Step 8: Kaggle Submission Preparation - COMPLETED")
print(f"\n🏆 SKLEARN LASSO REGRESSION MODEL READY!")
print(f"🎯 Optimal α: {model.alpha} | Training R²: {model.score(X_train_scaled, y_train):.4f}")
print(f"📊 Cross-validation R²: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print(f"🔍 Feature Selection: {np.sum(np.abs(model.coef_) >= 1e-6)}/{len(model.coef_)} active features ({np.sum(np.abs(model.coef_) < 1e-6)/len(model.coef_)*100:.1f}% sparsity)")


STEP 8: KAGGLE SUBMISSION PREPARATION
📁 LOADING TEST DATA AND MAKING PREDICTIONS:
--------------------------------------------------
Test data shape: (260, 80)
✅ Test predictions generated for 260 samples
Prediction range: [41185.43, 455742.31]
📤 SUBMISSION FILE CREATED:
--------------------------------------------------
File saved at: /Users/hemanthmada/vscodeProjects/ml_assignment_1/submissions/3_LASSO_regression_sklearn.csv
Submission shape: (260, 2)

First 5 predictions:
     Id     HotelValue
0   893  139107.786445
1  1106  303128.881744
2   414  121850.617881
3   523  169622.270272
4  1037  308451.176528

🎓 SKLEARN LASSO REGRESSION SUMMARY
✅ Step 1: Data Loading and Exploration - COMPLETED
✅ Step 2: Data Preprocessing and Feature Engineering - COMPLETED
✅ Step 3: Feature Scaling - COMPLETED
✅ Step 4: Sklearn LASSO Regression with CV - COMPLETED
✅ Step 5: Comprehensive Error Function Analysis - COMPLETED
✅ Step 6: Model Diagnostics and Validation - COMPLETED
✅ Step 7: LASSO Model